#  GPT-2 Generation and Fine-Tuning

This notebook explores GPT-2 (Generative Pretrained Transformer-2) from OpenAI. Read more about it [here](https://openai.com/blog/better-language-models/).

Activities include:

0. Setup
1. Generate samples from pre-trained gpt-2 model
2. Fine-tune gpt-2 on text of your choosing. 

Adapted by Robert Twomey (rtwomey@unl.edu) for Machine Learning for the Arts SP22 from this [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) by [Max Woolf](http://minimaxir.com). See his repo [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple).

In [1]:
# 0. Setup

Run once to install the library

In [2]:
%pip install -q gpt-2-simple

Note: you may need to restart the kernel to use updated packages.


restart the kernel and run the imports

In [3]:
from datetime import datetime

import gpt_2_simple as gpt2
import tensorflow as tf

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [4]:
!nvidia-smi

Mon Feb 21 17:51:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

note the memory usage (0MiB / 32510MiB) for the Tesla V100.
You can re-rerun the above cell to see what memory your code/models are using during training.

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are four released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the the current working directory at `/models/<model_name>`.

In [5]:
model_name = "355M"  # largest model we can fine-tune

run once to download the file

In [6]:
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 339Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.19Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 954Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:20, 69.5Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 817Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.22Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 5.41Mit/s]                                                      


# 1. Generate Text From The Pretrained Model

If you want to generate text from the pretrained model pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`. (This is currently the only way to generate text from the 774M or 1558M models with this notebook.)

In [7]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

2022-02-21 17:52:06.561981: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-21 17:52:07.745359: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading pretrained model models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


## Sample from the model
The follow cell samples from gpt-2, using the provided prefix (seed) and other parameters. It starts the TF session and generates the samples.

Try changing the parameters below to change the output: 
- `prefix` is the prompt. This will be the starting string/seed for your generation. Use your own text. 
- `temperature` sets the variability/randomness of the output. Range 0.0-1.0
- `length` sets the lenght of output (in tokens). max is 1024.


In [8]:
gpt2.generate(
    sess,
    model_name=model_name,
    prefix="The universe is a mysterious place full of",
    length=100,
    temperature=0.7,
    top_p=0.9,
    nsamples=5,
    batch_size=5,
)

The universe is a mysterious place full of possibilities, and it's these possibilities that are sometimes lost in the vastness of time. It's hard to grasp the depths of something so vast, and sometimes, that's all you can get. In the last two years, I've been lucky enough to have seen an incredible amount of stories about the concept of time travel, and I've noticed a common thread: they're all stories that deal with the notion of time travel. These stories are all about time travel being possible, but also about
The universe is a mysterious place full of mysterious things. How did we get here? How did we get to this place? How can we keep going?

My friend Eric is a scientist who has been working with me for a while. He has developed a method of measuring the length of time it takes for a single photon to travel through a closed loop and then through a closed loop to the outside world. He has called this process the "inverse Compton scattering" method. He has found that a photon trave

## Activities
- try varying the prefix. 
  - what length of prefix works best with the given model? 
  - how does the choice of prefix change the format/form of the output.
- try varying the temperature.
- try loading the different sized models (124M, 355M, 774M, 1558M) and generate text without changing the other parameters. 
  - Do you notice any qualitative differences in the output? 

# 2. Fine-Tuning GPT-2

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [9]:
# model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = "sess" in locals() or "sess" in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

2022-02-21 17:54:03.166347: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


## Upload a text file
For this, we will use a text file you provide to finetune (continue training) GPT-2. You can use any plain text (.txt) file. 

Simply drag and dropy our text file into the file browser at left. 

Once you have uploaded your file, update the file name in the cell below, then run it.

In [10]:
file_name = "recipes.txt"  # your file here

## Run the finetuning

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every `save_every` steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them. If your input text is smaller, training might proceed more quickly.

Other optional-but-helpful parameters for `gpt2.finetune`:

*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [11]:
gpt2.finetune(
    sess,
    dataset=file_name,
    model_name=model_name,
    steps=50,
    restore_from="fresh",  # change to 'latest' to resume
    run_name="run1",
    print_every=10,
    learning_rate=1e-5,
    sample_every=25,
    save_every=50,
)

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1



2022-02-21 17:54:48.772074: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  9.36it/s]

dataset has 24094 tokens
Training...


[10 | 26.90] loss=2.13 avg=2.13
[20 | 46.70] loss=1.90 avg=2.01
======== SAMPLE 1 ========
 to the same point.

I'm going to come back to this topic at the end of this episode (although it's important to note I will be reviewing an older episode of The Simpsons that was originally broadcast more than 30 years ago)… however, if you've gotten this far then you're going to appreciate the context of what I'm going to talk about. It will hopefully provide some insight into why I believe in Homer's belief in Santa Clause and why I think I have no idea what Christmas is.

I think it's important to clarify what this concept means today because it is a relatively new concept to us.

Back when I was a kid, I remember going to my first school reunion with my best friend's parents and saying to them, "Why are you teaching us to read? Who are you going to give an education to?" We got to tell you, I think the answer has to do with Santa Clause…

We all know that Santa Clause is one of the oldest Ch

## Notes on finetuning

Keep an eye on the loss, and how quickly it is dropping. A too-rapid drop in loss could be a sign of overfitting, and a learning rate (lr) that is too high. 

After the model is trained, you can download the checkpoint folder to save your work. Training checkpoints are saved to `checkpoint/run1` (or whatever you chose for the run name above).

You can compress it to a rar file and download that. Ask the instructor how.

You're done! Feel free to go to the Generate Text From The Trained Model section to generate text based on your retrained model.

## Finetune some more

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [12]:
# model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = "sess" in locals() or "sess" in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

2022-02-21 17:59:52.730023: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


To fine-tune some more, run the following. Be sure to increase the number of steps (if it was `500` before, change to `1000` to train for 500 more. the number is cumulative).

In [ ]:
gpt2.finetune(
    sess,
    dataset=file_name,
    model_name=model_name,
    steps=400,
    restore_from="latest",  # change to 'latest' to resume
    run_name="run1",
    print_every=10,
    learning_rate=1e-5,
    sample_every=100,
    save_every=100,
)

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1

Loading checkpoint checkpoint/run1/model-50
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-50
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  8.98it/s]

dataset has 24094 tokens
Training...


[60 | 28.76] loss=1.24 avg=1.24
[70 | 48.69] loss=1.06 avg=1.15
[80 | 68.62] loss=1.06 avg=1.12
[90 | 88.54] loss=0.90 avg=1.06
[100 | 108.47] loss=0.74 avg=1.00
Saving checkpoint/run1/model-100
======== SAMPLE 1 ========
 lightly, add half of the flour mixture. Roll the dough into a 12-in. rope. Place in a greased bowl; cover and let rise in a warm place until double in bulk, about 45 minutes. Spray a 10-in. springform pan 2 or 3/4 in. deep with cooking spray. Prepare filling. Combine flour, butter, sugar, salt and vanilla. Roll dough into a 12-in. rope; tie off and chill. Place 1/4 in. apart on prepared pan. Cover with filling; bake until golden brown, 22-25 minutes.

Beat egg and 2 Tbsp. canola oil in a small bowl until butter melts. Gradually beat in remaining 2 Tbsp. canola oil. Spread butter-drenched dough into 1-tablespoon glaze; allow to rest 10 minutes. Top with remaining dough; sprinkle with remaining canola oil.

Meanwhile, prepare filling. In a small saucepan over medium-hi

# 3. Generate Text From The Finetuned Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
# gpt2.generate(sess, run_name='run1') # no prefix, unconditional generation

In [14]:
gpt2.generate(sess, run_name="run1", prefix="Anakin had the high ground this time")

Anakin had the high ground this time, his lightsaber out.
PALPATINE
I told you, you would not defeat Count Dooku
with words alone!
ANAKIN
I know you do, Chancellor.
PALPATINE
I have relied on the teachings of your
Unified Force™ Sages, Anakin. They
have trained me in the ways of the dark
lightning, and I’ve become them.
ANAKIN
I wonder what it would be like to learn
from you yourself.
PALPATINE
You will learn much you will, Anakin.
You have always known how to act on impulse
and lose your temper, but you will learn
to control your emotions, to accept
whatever comes your way.
ANAKIN
I can see how it may be difficult.
PALPATINE
Your faith in your abilities is unwavering
I can assure you. But I do hope
that over the coming days and
weeks you will gain more self-discipline.
ANAKIN
Thank you, Chancellor.
PALPATINE
Are you ready, my apprentice?
EXT. CORUSCANT - SENATE CHAMBER
The WOOKIRETURER delivers the news to the Chancellor
and his Council.
ANAKIN
Yes, sir.
INT. LANDING PLATFORM - TREK -

## Notes
If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [15]:
gpt2.generate(
    sess, length=250, temperature=1.2, prefix="experiment", nsamples=5, batch_size=5
)

experiment suggests that animals make inferences more easily than humans do. What the research team found was that the chances that a shih tzus face would be curved toward the left were 2.4 times greater than those it would be to the right. This result clearly indicated that animals’ understanding of proportion was more primitive than that of logic.

Surrogate Mother Experiment
Study Conducted by: Harry Harlow and Rosalie Rayner
Study Conducted in 1954 at the University of Michigan

Experiment Details: The 1954 Surrogate Mother Experiment was one of the most controversial experiments in psychology. The experiment was designed to investigate the influence of early childhood development and the guidance systems of mother and infant.
Upon request of a media outlet for a story on the experiment, Jack Kevorkian of ABC News called into his show to talk about the experiment. During the interview Jack revealed that he and his camera crew were there to interview an unethical mother whose 14-mon

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [14]:
gen_file = "gpt2_gentext_{:%Y%m%d_%H%M%S}.txt".format(datetime.utcnow())

gpt2.generate_to_file(
    sess,
    destination_path=gen_file,
    length=500,
    temperature=1.2,
    nsamples=100,
    batch_size=20,
)

Download the file by hand in the browser at left.

## Load a Trained Model Checkpoint

Uploaded your saved checkpoint and unzip it.

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

This will reset or start the tensorflow session as necessary.

In [ ]:
tf.compat.v1.reset_default_graph()

if not sess:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

gpt2.load_gpt2(sess, run_name="run1")

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# License

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Reference
- Max's [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!
- Original repo: [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple) by [Max Woolf](http://minimaxir.com). 
- Original [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) from Max.